# Answer Coalescence

Strider finds and ranks all paths that fulfill the query.  Some of these answers can be very similar.  For instance, two answers may differ only by a gene node, but perhaps those genes are from the same family.  This permits a higher-level, more parsimonious description of an answer set.

In [1]:
import requests
import json
from notebook_functions import reasonerGraphToCytoscape, knowledgeGraphToCytoscape
from cyjupyter import Cytoscape
import time

In [2]:
def getlabel(curie):
    url = 'https://nodenormalization-sri.renci.org/get_normalized_nodes'
    r = requests.get(url,params={'curie':curie}).json()
    try:
        return r[curie]['id']['label']
    except:
        return ''

class Strider:
    def __init__(self):
        self.url='http://robokop.renci.org:5781'
    def call(self,question):
        message = {'message': {'query_graph': question}}
        return self.send_message(message)
    def send_message(self,message):
        response = requests.post(f'{self.url}/query',json=message)
        if response.status_code == 200:
            pid = response.json()
            return pid
        else:
            print(response.status_code)
            return None
    def query_result(self,pid):
        r = requests.get(f'{self.url}/results',params={'query_id':pid})
        return r.json()
    
strider = Strider()

In [3]:
def create_question(relation_type,disease_curie=None,chem_curie=None):
    question = { "nodes": [{"id":"n0", "type":"disease"},
                          {"id": "n1", "type":"chemical_substance"}],
                 "edges": [ {"id": "e0", "source_id": "n1", "target_id": "n0", "type": relation_type}]}
    if disease_curie is not None:
        question['nodes'][0]['curie']=disease_curie
    if chem_curie is not None:
        question['nodes'][1]['curie']=chem_curie
    return question

In [4]:
disease = "MONDO:0005148"   #T2D
edgetype="treats"
question = create_question(edgetype,disease_curie=disease)
message = {"message": {"query_graph": question}}

c=reasonerGraphToCytoscape(question)
Cytoscape(data=c, visual_style=c["style"], layout_name='grid', layout={"name": "grid", "height": "200px"})

Cytoscape(data={'elements': {'nodes': [{'data': {'id': 'n0', 'label': 'disease\n[MONDO:0005148]', 'curie': 'MO…

In [5]:
p = strider.send_message(message)
print(p)

cffad73d-1147-4930-ac24-2b464f5f5cb5


In [6]:
strider_result=strider.query_result(p)
while len(strider_result['results']) == 0:
    time.sleep(5)
    print('checking')
    strider_result = strider.query_result(p)
print('results found')

checking
results found


In [9]:
time.sleep(5)
strider_result = strider.query_result(p)
for node in strider_result['knowledge_graph']['nodes']:
    print(node['id'], node['name'])

MONDO:0005148 type 2 diabetes mellitus
PUBCHEM:3013166 L 783281
CHEBI:4806 (-)-epigallocatechin 3-gallate
CHEBI:9434 telmisartan
CHEBI:16118 berberine
MESH:C074031 [OBSOLETE] exenatide
CHEBI:50991 thiazolidenedione
PUBCHEM:6442063 Docosahexaenoic Acids
DRUGBANK:DB00284 Acarbose
PUBCHEM:667431 Gliclazide
CHEBI:5441 glyburide
CHEMBL.COMPOUND:CHEMBL1201174 SITAGLIPTIN
CHEBI:94686 orlistat
CHEBI:7986 Pentoxifylline
CHEBI:6801 metformin
CHEBI:47612 bezafibrate
CHEBI:28197 daidzein
CHEBI:26513 quinolines
CHEBI:4631 diosmin
CHEBI:4784 enalapril
CHEBI:6682 mangiferin
CHEBI:27999 tolbutamide
CHEBI:439329 (S)-fenfluramine
CHEBI:3650 chlorpropamide
CHEBI:28794 coumarin
PUBCHEM:23927 Lead
CHEBI:135285 vildagliptin
CHEBI:3962 curcumin
CHEBI:71271 saxagliptin hydrate
CHEBI:25681 omega-3 fatty acid
CHEBI:77148 ginsenoside Re
CHEMBL.COMPOUND:CHEMBL1829174 FASIGLIFAM
CHEBI:63631 topiramate
CHEBI:5384 glipizide
CHEBI:16731 (E)-cinnamaldehyde
CHEBI:16325 lithocholic acid
CHEBI:28088 genistein
CHEBI:40968

## Coalesce Answers by Property

In [11]:
url = 'https://answercoalesce.renci.org/coalesce'
params = {'method':'property'}
property_result = requests.post(url,json=strider_result,params=params)
print(property_result.status_code)

200


In [12]:
results = property_result.json()['results']
for result in results:
    chemnode = result['node_bindings'][1]
    print(len(chemnode['kg_id']), chemnode['p_values'], chemnode['properties'])

23 [1.4507062494232832e-32] ['drugbank.investigational']
22 [2.9927753494707155e-30] ['drugbank.approved']
20 [1.2767864105470421e-24] ['therapeutic_flag']
10 [4.823367861758246e-24] ['hypoglycemic_agent']
8 [3.3494981019524477e-22, 8.916362091726007e-21] ['Drugs Used in Diabetes', 'Blood Glucose Lowering Agents']
22 [4.830130548044178e-22, 6.478713505765552e-22] ['drug', 'pharmaceutical']
24 [6.205859186704515e-21] ['application']
7 [8.254275649747387e-20] ['Oral Hypoglycemics']
14 [1.0939886086070841e-19] ['oral']
10 [1.3742711531874345e-19] ['Alimentary Tract and Metabolism']
14 [3.4597402886307423e-16] ['inhibitor']
9 [5.4258882586528116e-15] ['Enzyme Inhibitors']
9 [6.668684135065894e-15] ['Cytochrome P-450 Enzyme Inhibitors']
4 [5.936897949610144e-14, 5.594130321207606e-13, 2.7511305706926877e-10, 2.6960183242478735e-08] ['Sulfonylurea Compounds', 'Sulfonylureas', 'Urea', 'Sulfones']
12 [1.0815168960134938e-13] ['enzyme_inhibitor']
29 [1.3741445773945114e-13] ['biological_role']


In [ ]:
#modify notebook functions to show an answer.  Show something reasonably sized, like the first answer with 5 compounds.

## Coalesce Answers by Ontology

In [23]:
url = 'https://answercoalesce.renci.org/coalesce'
params = {'method':'ontology'}
o_r = requests.post(url,json=strider_result,params=params)
print(o_r.status_code)
ontology_result = o_r.json()
print( len(ontology_result['results']))

200
1


In [25]:
results = ontology_result['results']
for result in results:
    print('================')
    chemnode = result['node_bindings'][1]
    sclass = chemnode['superclass']
    label = getlabel(sclass)
    nodes = chemnode['kg_id']
    labels = {x: f'{getlabel(x)}' for x in nodes }
    for x in nodes:
        print(f'{labels[x]} ({x})')
    print('----have superclass----')
    print(f'{label} ({sclass})')

chlorpropamide (CHEBI:3650)
tolbutamide (CHEBI:27999)
glipizide (CHEBI:5384)
glyburide (CHEBI:5441)
----have superclass----
N-sulfonylurea (CHEBI:76983)


In [16]:
#This is where it would be good to show an answer.  The KG coming back includes all the stuff that went in of course.
#c=knowledgeGraphToCytoscape(ontology_result.json()['knowledge_graph'])
#Cytoscape(data=c, visual_style=c["style"], layout_name='cose', layout={ "height": "200px"})

KeyError: 'name'

In [33]:
def create_pd_question(p_curie):
    question = { "nodes": [{"id":"n0", "type":"disease"},
                          {"id": "n1", "type":"phenotypic_feature","curie":p_curie}],
                 "edges": [ {"id": "e0", "source_id": "n0", "target_id": "n1", "type": "has_phenotype"}]}
    return question

In [35]:
p_curie = "HP:0012649"
question = create_pd_question(p_curie)
print(json.dumps(question,indent=4))
message = {"message": {"query_graph": question}}
metformin_p = strider.send_message(message)
strider_result=strider.query_result(metformin_p)
while len(strider_result['results']) == 0:
    time.sleep(5)
    strider_result = strider.query_result(metformin_p)
print(len(strider_result['results']))

{
    "nodes": [
        {
            "id": "n0",
            "type": "disease"
        },
        {
            "id": "n1",
            "type": "phenotypic_feature",
            "curie": "HP:0012649"
        }
    ],
    "edges": [
        {
            "id": "e0",
            "source_id": "n0",
            "target_id": "n1",
            "type": "has_phenotype"
        }
    ]
}


KeyboardInterrupt: 

In [ ]:
chem='CHEBI:28088'   #prednisone
edgetype="treats"
question = create_question(edgetype,chem_curie=chem)
message = {"message": {"query_graph": question}}

c=reasonerGraphToCytoscape(question)
Cytoscape(data=c, visual_style=c["style"], layout_name='grid', layout={"name": "grid", "height": "200px"})

In [ ]:
metformin_p = strider.send_message(message)
strider_result=strider.query_result(metformin_p)
while len(strider_result['results']) == 0:
    time.sleep(5)
    print('checking')
    strider_result = strider.query_result(metformin_p)
print('results found')

In [ ]:
strider_result = strider.query_result(metformin_p)
for n in strider_result['knowledge_graph']['nodes']:
    print(n['id'], n['name'])

In [ ]:
url = 'https://answercoalesce.renci.org/coalesce'
params = {'method':'ontology'}
ontology_result = requests.post(url,json=strider_result,params=params)
print(r.status_code)

In [ ]:
results = ontology_result.json()['results']
print(len(results))

In [ ]:
print(results)